In [1]:
import numpy as np
import pandas as pd

import tarfile
import gzip
import re
import os
import datetime as dt

In [2]:
year_num = 10 # Number of past years to consider
extremes_num = 10 # Number of hottest and coldest places to display

In [3]:
yearfiles = os.listdir("./gsod_all_years")
yearfiles.sort()
yearfiles = yearfiles[-year_num:]
years = [int(re.findall('\d+',yearfile)[0]) for yearfile in yearfiles]

In [4]:
station_loc = pd.read_csv('isd-history.csv')
station_loc = station_loc.replace([0.0, -999.0, -999.9],np.nan)
station_loc = station_loc[pd.notnull(station_loc['LAT']) & pd.notnull(station_loc['LON'])]
station_loc = station_loc[[int(re.findall('^\d{4}', str(end_year))[0])==max(years) for end_year in station_loc['END']]]
station_loc = station_loc[[int(re.findall('^\d{4}', str(beg_year))[0])<=min(years) for beg_year in station_loc['BEGIN']]]

In [5]:
station_loc['LBL'] = station_loc[['STATION NAME','STATE','CTRY']].apply(lambda x: x.str.cat(sep=', '), axis=1)
station_loc['ELEV_LBL'] = station_loc['ELEV(M)'].apply(lambda x: 'Elevation: '+str(x)+' m' if ~np.isnan(x) else np.nan)
station_loc['LBL'] = station_loc[['LBL','ELEV_LBL']].apply(lambda x: x.str.cat(sep='<br>'), axis=1)
station_loc = station_loc.drop(['STATION NAME','STATE','ELEV_LBL','ICAO','BEGIN','END'], axis=1)
#station_loc = station_loc.sample(stat_num)

In [6]:
df = pd.DataFrame([])
df_day = pd.DataFrame([])

def preprocess_station_file_content(content):
    headers=content.pop(0)
    headers=[headers[ind] for ind in [0,1,2,3,4,8,11,12]]
    for d in range(len(content)):
        content[d]=[content[d][ind] for ind in [0,1,2,3,5,13,17,18]]
    content=pd.DataFrame(content, columns=headers)
    content.rename(columns={'STN---': 'USAF'}, inplace=True)
    content['MAX'] = content['MAX'].apply(lambda x: re.sub("\*$","",x))
    content['MIN'] = content['MIN'].apply(lambda x: re.sub("\*$","",x))
    content[['WBAN','TEMP','DEWP','WDSP','MAX','MIN']] = content[['WBAN','TEMP','DEWP','WDSP','MAX','MIN']].apply(pd.to_numeric)
    content['YEARMODA']=pd.to_datetime(content['YEARMODA'], format='%Y%m%d', errors='ignore')
    content['YEAR']=pd.DatetimeIndex(content['YEARMODA']).year
    content['MONTH']=pd.DatetimeIndex(content['YEARMODA']).month
    content['DAY']=pd.DatetimeIndex(content['YEARMODA']).day
    return content

In [7]:
# yearfile = yearfiles[-1]
# print(yearfile)
# i=0
# tar = tarfile.open("./gsod_all_years/"+yearfile, "r")
# print(len(tar.getmembers()[1:]))
# #for member in np.random.choice(tar.getmembers()[1:], size=stat_num, replace=False):
# for member in tar.getmembers()[1:]:
#     name_parts = re.sub("\.op\.gz$","",re.sub("^\./","",member.name)).split("-")
#     usaf = name_parts[0]
#     wban = int(name_parts[1])
#     if station_loc[(station_loc['USAF']==usaf) & (station_loc['WBAN']==wban)].shape[0]!=0:
#         i=i+1
#         #if i%(stat_num//10) == 0: print(i)
#         f=tar.extractfile(member)
#         f=gzip.open(f, 'rb')
#         content=[re.sub(" +", ",", line.decode("utf-8")).split(",") for line in f.readlines()]
#         content=preprocess_station_file_content(content)
#         df_day = df_day.append(content[content['YEARMODA']==content['YEARMODA'].max()])
#         content = content.groupby(['USAF','WBAN','YEAR','MONTH']).agg('median').reset_index()
#         df = df.append(content)
# tar.close()

In [8]:
# df

In [9]:
# station_loc

In [10]:
# merged_df = pd.merge(df, station_loc, how="right", on="USAF")
# merged_df

In [11]:
# merged_clean_df = merged_df.loc[merged_df["USAF"] != "999999", :]
# merged_clean_df

In [12]:
# merged_clean_df.to_csv('merged_cleaned_dataframe.csv',index=False)

In [13]:
# day = df_day['YEARMODA'].max()
# df_day = df_day[df_day['YEARMODA']==day]

In [14]:
for yearfile in yearfiles[:-1]:
    print(yearfile)
    i=0
    tar = tarfile.open("./gsod_all_years/"+yearfile, "r")
    print(len(tar.getmembers()[1:]))
    #for member in np.random.choice(tar.getmembers()[1:], size=stat_num, replace=False):
    for member in tar.getmembers()[1:]:
        name_parts = re.sub("\.op\.gz$","",re.sub("^\./","",member.name)).split("-")
        usaf = name_parts[0]
        wban = int(name_parts[1])
        if station_loc[(station_loc['USAF']==usaf) & (station_loc['WBAN']==wban)].shape[0]!=0:
            i=i+1
            #if i%(stat_num//10) == 0: print(i)
            f=tar.extractfile(member)
            f=gzip.open(f, 'rb')
            content=[re.sub(" +", ",", line.decode("utf-8")).split(",") for line in f.readlines()]
            content=preprocess_station_file_content(content)
            df_day = df_day.append(content[content['YEARMODA']==content['YEARMODA'].max()])
            content = content.groupby(['USAF','WBAN','YEAR','MONTH']).agg('median').reset_index()
            years_df = df.append(content)
    tar.close()

gsod_2010.tar
11238
gsod_2011.tar
10702
gsod_2012.tar
11517
gsod_2013.tar
11580
gsod_2014.tar
11540
gsod_2015.tar
11664
gsod_2016.tar
12076
gsod_2017.tar
11923
gsod_2018.tar
12875


In [15]:
df_day

,USAF,WBAN,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY
364,161490,99999,2010-12-31,40.7,38.8,1.8,42.8,38.8,2010,12,31
364,723759,53990,2010-12-31,63.2,50.3,11.3,66.2,50.0,2010,12,31
364,162420,99999,2010-12-31,45.4,38.7,4.9,50.0,38.1,2010,12,31
364,478400,99999,2010-12-31,34.1,20.2,17.1,39.2,30.2,2010,12,31
364,723555,93940,2010-12-31,56.1,49.3,8.8,62.6,46.4,2010,12,31
...,...,...,...,...,...,...,...,...,...,...,...
360,717270,99999,2018-12-31,9.7,5.1,1.9,19.4,1.0,2018,12,31
209,355220,99999,2018-12-31,18.9,15.4,2.4,21.6,11.8,2018,12,31
208,355630,99999,2018-12-31,2.0,-3.7,999.9,11.8,-4.2,2018,12,31
358,131160,99999,2018-12-31,44.5,31.1,6.8,50.0,36.1,2018,12,31


In [16]:
station_loc

,USAF,WBAN,CTRY,LAT,LON,ELEV(M),LBL
11,010010,99999,NO,70.933,-8.667,9.0,"JAN MAYEN(NOR-NAVY), NO<br>Elevation: 9.0 m"
13,010014,99999,NO,59.792,5.341,48.8,"SORSTOKKEN, NO<br>Elevation: 48.8 m"
17,010020,99999,NO,80.050,16.250,8.0,"VERLEGENHUKEN, NO<br>Elevation: 8.0 m"
18,010030,99999,NO,77.000,15.500,12.0,"HORNSUND, NO<br>Elevation: 12.0 m"
21,010060,99999,NO,78.250,22.817,14.0,"EDGEOYA, NO<br>Elevation: 14.0 m"
...,...,...,...,...,...,...,...
29749,A00019,53969,US,34.226,-99.284,385.6,"WILBARGER COUNTY AIRPORT, TX, US<br>Elevation:..."
29750,A00021,63884,US,37.601,-81.559,543.5,"KEE FIELD AIRPORT, WV, US<br>Elevation: 543.5 m"
29752,A00023,63890,US,30.350,-81.883,30.2,"WHITEHOUSE NAVAL OUTLYING FIELD, FL, US<br>Ele..."
29757,A00031,3725,US,38.994,-79.144,292.6,"GRANT COUNT AIRPORT, WV, US<br>Elevation: 292.6 m"


In [17]:
merged_df = pd.merge(df_day, station_loc, how="outer", on="USAF")
merged_df

,USAF,WBAN_x,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY,WBAN_y,CTRY,LAT,LON,ELEV(M),LBL
0,161490,99999.0,2010-12-31,40.7,38.8,1.8,42.8,38.8,2010.0,12.0,31.0,99999,IT,44.020,12.612,12.5,"RIMINI, IT<br>Elevation: 12.5 m"
1,161490,99999.0,2011-12-31,38.2,28.5,2.1,50.0,29.7,2011.0,12.0,31.0,99999,IT,44.020,12.612,12.5,"RIMINI, IT<br>Elevation: 12.5 m"
2,161490,99999.0,2012-12-31,34.7,32.3,2.2,46.8,26.6,2012.0,12.0,31.0,99999,IT,44.020,12.612,12.5,"RIMINI, IT<br>Elevation: 12.5 m"
3,161490,99999.0,2013-12-31,46.2,41.5,3.9,50.0,42.8,2013.0,12.0,31.0,99999,IT,44.020,12.612,12.5,"RIMINI, IT<br>Elevation: 12.5 m"
4,161490,99999.0,2014-12-31,38.1,25.4,15.6,41.0,32.0,2014.0,12.0,31.0,99999,IT,44.020,12.612,12.5,"RIMINI, IT<br>Elevation: 12.5 m"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302206,A00019,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53969,US,34.226,-99.284,385.6,"WILBARGER COUNTY AIRPORT, TX, US<br>Elevation:..."
302207,A00021,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63884,US,37.601,-81.559,543.5,"KEE FIELD AIRPORT, WV, US<br>Elevation: 543.5 m"
302208,A00023,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63890,US,30.350,-81.883,30.2,"WHITEHOUSE NAVAL OUTLYING FIELD, FL, US<br>Ele..."
302209,A00031,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3725,US,38.994,-79.144,292.6,"GRANT COUNT AIRPORT, WV, US<br>Elevation: 292.6 m"


In [18]:
merged_clean_df = merged_df.loc[merged_df["USAF"] != "999999", :]
merged_clean_df.dropna(how='any')
us_merged_df = merged_clean_df.loc[merged_df["CTRY"] == "US", :]
us_merged_df

,USAF,WBAN_x,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY,WBAN_y,CTRY,LAT,LON,ELEV(M),LBL
9,723759,53990.0,2010-12-31,63.2,50.3,11.3,66.2,50.0,2010.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
10,723759,53990.0,2011-12-31,48.6,38.3,5.9,68.0,32.0,2011.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
11,723759,53990.0,2012-12-31,37.6,30.9,5.6,41.0,35.6,2012.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
12,723759,53990.0,2013-12-31,32.9,23.3,2.4,51.8,21.2,2013.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
13,723759,53990.0,2014-12-31,34.5,25.8,3.4,39.2,30.2,2014.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302206,A00019,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53969,US,34.226,-99.284,385.6,"WILBARGER COUNTY AIRPORT, TX, US<br>Elevation:..."
302207,A00021,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63884,US,37.601,-81.559,543.5,"KEE FIELD AIRPORT, WV, US<br>Elevation: 543.5 m"
302208,A00023,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63890,US,30.350,-81.883,30.2,"WHITEHOUSE NAVAL OUTLYING FIELD, FL, US<br>Ele..."
302209,A00031,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3725,US,38.994,-79.144,292.6,"GRANT COUNT AIRPORT, WV, US<br>Elevation: 292.6 m"


In [19]:
new_df=us_merged_df.dropna(how='any')

In [20]:
new_df

,USAF,WBAN_x,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY,WBAN_y,CTRY,LAT,LON,ELEV(M),LBL
9,723759,53990.0,2010-12-31,63.2,50.3,11.3,66.2,50.0,2010.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
10,723759,53990.0,2011-12-31,48.6,38.3,5.9,68.0,32.0,2011.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
11,723759,53990.0,2012-12-31,37.6,30.9,5.6,41.0,35.6,2012.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
12,723759,53990.0,2013-12-31,32.9,23.3,2.4,51.8,21.2,2013.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
13,723759,53990.0,2014-12-31,34.5,25.8,3.4,39.2,30.2,2014.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301564,724856,99999.0,2017-12-31,35.1,13.4,2.8,46.4,26.6,2017.0,12.0,31.0,99999,US,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."
301565,724856,99999.0,2018-12-31,32.1,19.8,11.0,41.0,24.8,2018.0,12.0,31.0,99999,US,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."
301599,725835,24119.0,2017-12-31,36.7,21.4,3.7,53.6,23.0,2017.0,12.0,31.0,24119,US,40.612,-116.892,1373.1,"BATTLE MOUNTAIN AIRPORT, NV, US<br>Elevation: ..."
301600,725835,24119.0,2018-12-31,27.9,21.5,7.5,33.8,19.4,2018.0,12.0,31.0,24119,US,40.612,-116.892,1373.1,"BATTLE MOUNTAIN AIRPORT, NV, US<br>Elevation: ..."


In [21]:
# us_drop_na_df = us_merged_df.dropna(how='any')
# us_drop_na_df

In [22]:
new_df

,USAF,WBAN_x,YEARMODA,TEMP,DEWP,WDSP,MAX,MIN,YEAR,MONTH,DAY,WBAN_y,CTRY,LAT,LON,ELEV(M),LBL
9,723759,53990.0,2010-12-31,63.2,50.3,11.3,66.2,50.0,2010.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
10,723759,53990.0,2011-12-31,48.6,38.3,5.9,68.0,32.0,2011.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
11,723759,53990.0,2012-12-31,37.6,30.9,5.6,41.0,35.6,2012.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
12,723759,53990.0,2013-12-31,32.9,23.3,2.4,51.8,21.2,2013.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
13,723759,53990.0,2014-12-31,34.5,25.8,3.4,39.2,30.2,2014.0,12.0,31.0,53990,US,33.909,-94.859,143.9,"MC CURTAIN COUNTY REGIONAL AIRPORT, OK, US<br>..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301564,724856,99999.0,2017-12-31,35.1,13.4,2.8,46.4,26.6,2017.0,12.0,31.0,99999,US,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."
301565,724856,99999.0,2018-12-31,32.1,19.8,11.0,41.0,24.8,2018.0,12.0,31.0,99999,US,38.544,-118.634,1284.7,"HAWTHORNE INDUSTRIAL, NV, US<br>Elevation: 128..."
301599,725835,24119.0,2017-12-31,36.7,21.4,3.7,53.6,23.0,2017.0,12.0,31.0,24119,US,40.612,-116.892,1373.1,"BATTLE MOUNTAIN AIRPORT, NV, US<br>Elevation: ..."
301600,725835,24119.0,2018-12-31,27.9,21.5,7.5,33.8,19.4,2018.0,12.0,31.0,24119,US,40.612,-116.892,1373.1,"BATTLE MOUNTAIN AIRPORT, NV, US<br>Elevation: ..."


In [36]:
for index, row in new_df.iterrows():
    LBL = row[16]
    US = LBL.find("US")
    state = LBL[(US-4)] + LBL[(US-3)]
    print(state)

OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
AR
AR
AR
AR
AR
AR
AR
AR
AR
NC
NC
NC
NC
NC
NC
NC
NC
NC
GA
GA
GA
GA
GA
GA
GA
GA
GA
CA
CA
CA
CA
CA
CA
CA
CA
CA
LA
LA
LA
LA
LA
LA
LA
LA
NM
NM
NM
NM
NM
NM
NM
NM
NM
TX
TX
TX
TX
TX
TX
TX
TX
TX
MS
MS
MS
MS
MS
MS
MS
MS
MS
TX
TX
TX
TX
TX
TX
TX
TX
TX
AL
AL
AL
AL
AL
AL
AL
AL
AL
NC
NC
NC
NC
NC
NC
NC
NC
NC
MN
MN
MN
MN
MN
MN
MN
MN
MN
TX
TX
TX
TX
TX
TX
TX
TX
TX
GA
GA
GA
GA
GA
GA
GA
GA
GA
GA
GA
GA
GA
GA
GA
GA
GA
GA
TX
TX
TX
TX
TX
TX
TX
TX
TX
AR
AR
AR
AR
AR
AR
AR
AR
AR
OK
OK
OK
OK
OK
OK
OK
OK
OK
NM
NM
NM
NM
NM
NM
NM
NM
NM
LA
LA
LA
LA
LA
LA
LA
LA
LA
TX
TX
TX
TX
TX
TX
TX
TX
TX
SC
SC
SC
SC
SC
SC
SC
SC
SC
SC
SC
SC
SC
SC
SC
SC
SC
SC
CO
CO
CO
CO
CO
CO
CO
CO
CO
2 
2 
2 
2 
2 
2 
2 
2 
2 
AL
AL
AL
AL
AL
AL
AL
AL
AL
LU
LU
LU
LU
LU
LU
LU
LU
LU
TX
TX
TX
TX
TX
TX
TX
TX
TX
GA
GA
GA
GA
GA
GA
GA
GA
GA
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
HI
H

TX
TX
TX
TX
TX
TX
TX
TX
TX
SD
SD
SD
SD
SD
SD
SD
SD
SD
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
FL
FL
FL
FL
FL
FL
FL
FL
FL
GA
GA
GA
GA
GA
GA
GA
GA
GA
GA
GA
GA
GA
GA
GA
GA
GA
GA
MN
MN
MN
MN
MN
MN
MN
MN
MN
ME
ME
ME
ME
ME
ME
ME
ME
ME
MN
MN
MN
MN
MN
MN
MN
MN
MN
 m
 m
 m
 m
 m
 m
 m
 m
 m
WI
WI
WI
WI
WI
WI
WI
WI
WI
WI
WI
WI
WI
WI
WI
WI
WI
WI
CA
CA
CA
CA
CA
CA
CA
CA
CA
CO
CO
CO
CO
CO
CO
CO
CO
CO
NV
NV
NV
NV
NV
NV
NV
NV
NV
IA
IA
IA
IA
IA
IA
IA
IA
IA
NE
NE
NE
NE
NE
NE
NE
NE
 I
 I
 I
 I
 I
 I
 I
 I
 I
IA
IA
IA
IA
IA
IA
IA
IA
IA
IL
IL
IL
IL
IL
IL
IL
IL
IL
OH
OH
OH
OH
OH
OH
OH
OH
OH
OH
OH
OH
OH
OH
OH
OH
OH
OH
PA
PA
PA
PA
PA
PA
PA
PA
PA
PA
PA
PA
PA
PA
PA
PA
PA
PA
CO
CO
CO
CO
CO
CO
CO
CO
CO
CO
CO
CO
CO
CO
CO
CO
CO
CO
IL
IL
IL
IL
IL
IL
IL
IL
IL
MO
MO
MO
MO
MO
MO
MO
MO
MO
NJ
NJ
NJ
NJ
NJ
NJ
NJ
NJ
NJ
NM
NM
NM
NM
NM
NM
NM
NM
NM
MS
MS
MS
MS
MS
MS
MS
MS
MS
TN
TN
TN
TN
TN
TN
TN
TN
TN
SC
SC
SC
SC
SC
SC
SC
SC
SC
CA
CA
CA
CA
CA
CA
CA
CA
CA
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
LA
L

WI
WI
WI
WI
WI
WI
WI
WI
CO
CO
CO
CO
CO
CO
CO
CO
CO
ID
ID
ID
ID
ID
ID
ID
ID
ID
IA
IA
IA
IA
IA
IA
IA
IA
IA
PA
PA
PA
PA
PA
PA
PA
PA
PA
IA
IA
IA
IA
IA
IA
IA
IA
IA
CO
CO
CO
CO
CO
CO
CO
CO
CO
NV
NV
NV
NV
NV
NV
NV
NV
NV
CT
CT
CT
CT
CT
CT
CT
CT
CT
MA
MA
MA
MA
MA
MA
MA
MA
MA
KS
KS
KS
KS
KS
KS
KS
KS
KS
IN
IN
IN
IN
IN
IN
IN
IN
IN
VA
VA
VA
VA
VA
VA
VA
VA
VA
WV
WV
WV
WV
WV
WV
WV
WV
WV
TN
TN
TN
TN
TN
TN
TN
TN
TN
AR
AR
AR
AR
AR
AR
AR
AR
AR
MO
MO
MO
MO
MO
MO
MO
MO
MO
NC
NC
NC
NC
NC
NC
NC
NC
NC
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
NC
NC
NC
NC
NC
NC
NC
NC
NC
NC
NC
NC
NC
NC
NC
NC
NC
NC
VA
VA
VA
VA
VA
VA
VA
VA
VA
SC
SC
SC
SC
SC
SC
SC
SC
SC
MI
MI
MI
MI
MI
MI
MI
MI
MI
TX
TX
TX
TX
TX
TX
TX
TX
TX
LA
LA
LA
LA
LA
LA
LA
LA
LA
AL
AL
AL
AL
AL
AL
AL
AL
AL
FL
FL
FL
FL
FL
FL
FL
FL
FL
WI
WI
WI
WI
WI
WI
WI
WI
WI
IL
IL
IL
IL
IL
IL
IL
IL
IL
VA
VA
VA
VA
VA
VA
VA
VA
VA
WI
WI
WI
WI
WI
WI
WI
WI
WI
FL
FL
FL
FL
FL
FL
FL
FL
FL
TX
TX
TX
TX
TX
TX
TX
TX
TX
IN
IN
IN
IN
IN
IN
IN
IN
IN
SC
SC
SC
SC
SC
SC
SC
SC
SC
NJ
NJ
NJ
NJ
NJ
NJ
NJ
NJ
NJ
FL
F

MN
MN
MN
MN
MN
WI
WI
WI
WI
WI
WI
WI
WI
WI
WI
WI
WI
WI
WI
WI
WI
WI
WI
NE
NE
NE
NE
NE
NE
NE
NE
NE
AL
AL
AL
AL
AL
AL
AL
AL
AL
OK
OK
OK
OK
OK
OK
OK
OK
OK
FL
FL
FL
FL
FL
FL
FL
FL
FL
GA
GA
GA
GA
GA
GA
GA
GA
GA
OK
OK
OK
OK
OK
OK
OK
OK
OK
FL
FL
FL
FL
FL
FL
FL
FL
FL
OK
OK
OK
OK
OK
OK
OK
OK
OK
LA
LA
LA
LA
LA
LA
LA
LA
LA
WY
WY
WY
WY
WY
WY
WY
WY
WY
SC
SC
SC
SC
SC
SC
SC
SC
SC
CA
CA
CA
CA
CA
CA
CA
CA
CA
9 
9 
9 
9 
9 
9 
9 
9 
9 
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
AK
HI
HI
HI
HI
HI
HI
HI
HI
HI
MA
MA
MA
MA
MA
MA
MA
MA
MA
NY
NY
NY
NY
NY
NY
NY
NY
NY
CA
CA
CA
CA
CA
CA
CA
CA
CA
GA
GA
GA
GA
GA
GA
GA
GA
GA
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
MT
ND
ND
ND
ND
ND
ND
ND
ND
ND
MN
MN
MN
MN
MN
MN
MN
MN
MN
MI
MI
MI
MI
MI
MI
MI
MI
MI
ID
ID
ID
ID
ID
ID
ID
ID
ID
WY
WY
WY
WY
WY
WY
WY
WY
WY
MN
MN
MN
MN
MN
MN
MN
MN
MN
MN
MN
MN
MN
MN
MN
MN
MN
MN
ME
ME
ME
ME
ME
ME
ME
ME
ME
MI
MI
MI
MI
MI
MI
MI
MI
MI
MI
MI
MI
MI
MI
MI
MI
MI
MI
WI
WI
WI
WI
WI
WI
WI
WI
WI
MN
MN
MN
MN
MN
M

TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
GA
GA
GA
GA
GA
GA
GA
GA
GA
NC
NC
NC
NC
NC
NC
NC
NC
NC
LA
LA
LA
LA
LA
LA
LA
LA
LA
GA
GA
GA
GA
GA
GA
GA
GA
GA
AR
AR
AR
AR
AR
AR
AR
AR
AR
CO
CO
CO
CO
CO
CO
CO
CO
CO
FL
FL
FL
FL
FL
FL
FL
FL
FL
FL
FL
FL
FL
FL
FL
FL
FL
FL
CA
CA
CA
CA
CA
CA
CA
CA
CA
MO
MO
MO
MO
MO
MO
MO
MO
MO
CA
CA
CA
CA
CA
CA
CA
CA
CA
OH
OH
OH
OH
OH
OH
OH
OH
OH
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
TX
WA
WA
WA
WA
WA
WA
WA
WA
WA
TX
TX
TX
TX
TX
TX
TX
TX
TX
AK
AK
AK
AK
AK
AK
AK
AK
GA
GA
GA
GA
GA
GA
GA
GA
GA
IL
IL
IL
IL
IL
IL
IL
IL
IL
WA
WA
WA
WA
WA
WA
WA
WA
WA
MN
MN
MN
MN
MN
MN
MN
MN
MN
MT
MT
MT
MT
MT
MT
MT
MT
WA
WA
WA
WA
WA
WA
WA
WA
WA
WY
WY
WY
WY
WY
WY
WY
WY
ID
ID
ID
ID
ID
ID
ID
ID
ID
OR
OR
OR
OR
OR
OR
OR
OR
OR
MI
MI
MI
MI
MI
MI
MI
MI
MI
MN
MN
MN
MN
MN
MN
MN
MN
MN
WI
WI
WI
WI
WI
WI
WI
WI
WI
MN
MN
MN
MN
MN
MN
MN
MN
MN
MN
MN
MN
MN
MN
MN
MN
MN
MN
WI
WI
WI
WI
WI
WI
WI
WI
WI
NH
N

KY
KY
KY
KY
KY
KY
WV
WV
WV
WV
WV
WV
WV
WV
WV
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
MD
MD
MD
MD
MD
MD
MD
MD
MD
VA
VA
VA
VA
VA
VA
VA
VA
VA
TX
TX
TX
TX
TX
TX
TX
TX
TX
SC
SC
SC
SC
SC
SC
SC
SC
0 
0 
0 
0 
0 
0 
0 
0 
KY
KY
KY
KY
KY
KY
KY
KY
CA
CA
CA
CA
CA
CA
CA
CA
GA
GA
GA
GA
GA
GA
GA
GA
TX
TX
TX
TX
TX
TX
TX
TX
SC
SC
SC
SC
SC
SC
SC
SC
WY
WY
WY
WY
WY
WY
WY
NE
NE
NE
NE
NE
NE
NE
NE
AK
AK
AK
AK
AK
AK
AK
AK
IA
IA
IA
IA
IA
IA
IA
IA
VA
VA
VA
VA
VA
VA
VA
VA
LA
LA
LA
LA
LA
LA
LA
LA
 m
 m
 m
 m
 m
 m
 m
 m
OK
OK
OK
OK
OK
OK
OK
OK
IA
IA
IA
IA
IA
IA
IA
IA
TX
TX
TX
TX
TX
TX
TX
TX
AK
AK
AK
AK
AK
AK
AK
AK
NM
NM
NM
NM
NM
NM
NM
NM
TX
TX
TX
TX
TX
TX
TX
TX
ID
ID
ID
ID
ID
ID
ID
ID
LA
LA
LA
LA
LA
LA
LA
LA
SC
SC
SC
SC
SC
SC
SC
SC
 m
 m
 m
 m
 m
 m
 m
 m
KY
KY
KY
KY
KY
KY
KY
KY
CA
CA
CA
CA
CA
CA
CA
CA
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
WI
WI
WI
WI
WI
WI
WI
WI
VT
VT
VT
VT
VT
VT
VT
VT
CO
CO
CO
CO
CO
CO
CO
CO
SC
SC
SC
SC
SC
SC
SC
SC
FL
FL
FL
FL
FL
FL
FL
FL
MO
MO
MO
MO
MO
MO
MO
MO
MD
MD
MD
MD
MD
MD
MD
MD
CO
CO
CO
CO
CO
CO
CO
CO
1 
1 
1 
1

In [ ]:
# us_drop_na_df['YEARMODA']

In [ ]:
# list(us_drop_na_df['YEARMODA'])

In [ ]:
list(new_df['YEARMODA'])

In [ ]:

new_df.dtypes

In [ ]:
# from pymongo import MongoClient
# client = MongoClient('localhost', 27017)
# db = client.USWeather
# collection = db.weather
# db.collection.insert_many(new_df.to_dict('records'))